# Gemini ve LangChain ile LLM API'larını Çağırma Giriş 🦜🔗

Bu notebook'ta LangChain aracılığıyla LLM API'larını nasıl kullanacağınızı öğreneceksiniz. Örnek olarak Google'ın Gemini API'sını kullanacağız. Bu notebook'un sonunda, LangChain kullanarak API çağrıları yapmayı ve bunu neden yaptığımızı bileceksiniz.

## ⚙️ Kurulum

👉 Kurulum aşamasında oluşturduğumuz `.env` dosyasındaki ortam değişkenlerini yüklemek için aşağıdaki hücreyi çalıştırın:

In [1]:
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

True

👉 Hücrenin çıktısı "`True`" mu? Harika! Artık Gemini API ile kimlik doğrulaması yapmak için kullanılacak bir `GOOGLE_API_KEY` ortam değişkeni kurmuş olduk.

Eğer değilse, yardım isteyin.

## Basit Bir API Çağrısı Yapma

Bu notebook'ta şunların nasıl yapılacağını göstereceğiz:
1. Google'ın kendi kütüphanesini kullanarak API çağrısı yapma.
2. Aynı işlemi LangChain kullanarak yapma.

## Google Generative AI Kütüphanesini Kullanma

In [2]:
from google import genai

In [3]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="What is the capital of France?",
)

`response` nesnesine bir göz atalım.

In [4]:
response.candidates[0].content.parts[0].text

'The capital of France is **Paris**.'

Gerçek cevabı nasıl alabileceğinizi görüyor musunuz?

Neyse ki, cevabı hemen almak için sadece `.text` özelliğini kullanabiliriz. Deneyin.

In [5]:
response.text

'The capital of France is **Paris**.'

Gemini cevaplarını Markdown formatında döndürür. Bunu kullanalım!

In [6]:
from IPython.display import Markdown
Markdown(response.text)

The capital of France is **Paris**.

Oluşturma parametrelerini de değiştirebilirsiniz. `google.genai` kullanarak bunu şu şekilde yaparsınız:

In [7]:
from google import genai
from google.genai import types # We need to import types for the config

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Write a social media post about how much you're learning about transformers.",
    config=types.GenerateContentConfig(
        max_output_tokens=200,
        temperature=1.0
    )
)

In [8]:
Markdown(response.text)

Here are a few options for a social media post about learning about transformers, ranging in tone and focus. Choose the one that best fits your style!

**Option 1: Enthusiastic & Broad**

> My brain is officially buzzing with transformers! 🤯 From the self-attention mechanism to the encoder-decoder architecture, I'm absolutely fascinated by how these models work and the incredible things they can do. It's like unlocking a new superpower in the world of AI! So much to learn, so little time! #Transformers #AI #MachineLearning #DeepLearning #TechLearning #NLP

**Option 2: Slightly More Technical & Focused**

> Diving deep into the world of transformer models lately! 🤓 Really starting to wrap my head around concepts like positional encodings, multi-head attention, and why they've revolutionized NLP. It's a challenging but incredibly rewarding journey. Excited to see where this knowledge takes me! #TransformersAI #AttentionIsAll

Harika. Ancak başka bir API denemek istediğinizi düşünün, örneğin OpenAI'nin veya Anthropic'in?

Onların dokümantasyonlarını incelemek ve tüm kodunuzu onların API'sini kullanacak şekilde yeniden yazmak zorunda kalırsınız. Tabii ki benzer olacaktır, ancak aynı olmayacaktır.

Neyse ki LangChain var!

## LangChain Kullanma 🦜🔗

Neden LangChain kullanırsınız?

1. **Model-Bağımsız Kod**

   LangChain, farklı LLM sağlayıcıları (Google, OpenAI, Anthropic, vb.) arasında minimal kod değişikliği ile geçiş yapmanızı sağlayan soyutlamalar sunar. Google API'sine doğrudan kod yazarsanız, sağlayıcı değiştirmek önemli ölçüde yeniden düzenleme gerektirir.

2. **Birleşik Arayüz**

   LangChain, altta yatan API'den bağımsız olarak farklı LLM sağlayıcıları arasında etkileşimleri standartlaştırır ve tutarlı yöntemler ile yanıt formatları sunar.

3. **Bileşenlerle Çalışabilirlik**

   LangChain'in zincir ve pipeline mimarisi, tüm alt yapıyı kendiniz halletmeden prompt, bellek ve erişim sistemlerini birleştiren karmaşık iş akışları oluşturmayı kolaylaştırır.

4. **Yerleşik Araçlar**

   LangChain, çıktı ayrıştırma, prompt şablonları ve kendiniz uygulamanız gereken diğer yardımcı araçları içerir.

[LangChain'in chat entegrasyonları listesi](https://docs.langchain.com/oss/python/integrations/chat)'ne gidin ve entegrasyon listesine bakın. Favori LLM sağlayıcınızı bulabiliyor musunuz?

Kodumuzda `chat_models.ChatGoogleGenerativeAI` kullanmak istemiyoruz çünkü bu özellikle Gemini için yapılmış. LLM'yi değiştirmek istersek, modeli başlatma şeklimizi değiştirmek zorunda kalırız. Neyse ki LangChain bir modeli başlatmak için daha genel bir yol sunar.

Gemini'yi tekrar kullanalım, ancak şimdi LangChain'in genel Chat Models'ini kullanarak.

👉 [LangChain'in "Models" dokümantasyonu](https://docs.langchain.com/oss/python/langchain/models) sayfasına gidin ve Gemini kullanarak bir chat modelinin nasıl başlatılacağını bulun.

İpuçları:
1. Hemen "Basic Usage" bölümüne gidin.
2. Kullanmak istediğiniz modeli seçerek doğru dokümantasyonu hemen görebilirsiniz.

In [9]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash-lite", model_provider="google_genai")

Modelin en temel kullanımı sadece `.invoke()` metodunu kullanmaktır:

In [10]:
response = model.invoke("What is the capital of France")

Yanıta bir göz atalım. Nesnenin tüm öznitelik ve metodlarını içeren `__dict__`'ini güzel şekilde yazdırmak için `pprint()` kullanıyoruz.

In [11]:
from pprint import pprint
pprint(response.__dict__)

{'additional_kwargs': {},
 'content': 'The capital of France is **Paris**.',
 'id': 'lc_run--019c2550-2fed-7b80-9e5b-7424157f37a8-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'STOP',
                       'model_name': 'gemini-2.5-flash-lite',
                       'model_provider': 'google_genai',
                       'safety_ratings': []},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_token_details': {'cache_read': 0},
                    'input_tokens': 6,
                    'output_tokens': 8,
                    'total_tokens': 14}}


Cevabı çıkarın ve görüntüleyin. Markdown formatında olduğunu unutmayın, bu yüzden güzel görünmesini sağlayabilirsiniz.

In [12]:
Markdown(response.content)

The capital of France is **Paris**.

Modelin temperature değerini `.temperature` özniteliğine erişerek kontrol edebilirsiniz. Deneyin:

In [13]:
model.temperature, model.max_output_tokens

(0.7, None)

Modeli kullanmadan önce, özniteliklere yeni değerler atayarak oluşturma parametrelerini de ayarlayabiliriz.

Daha önce Google'ın kütüphanesini kullanarak sosyal medya gönderisi yazmak için yaptığımızın eşdeğerini kodlamaya çalışın.

> _Not_: Normal olarak modelin `max_output_tokens` değerini ayarlayabilmemiz gerekir (modeli başlatırken veya daha sonra özniteliği değiştirerek). _langchain_google_genai_'nin mevcut sürümü (4.1.1) bir [hataya](https://github.com/langchain-ai/langchain-google/issues/1454) sahip ve bu çalışmıyor. Geçici çözüm? `max_output_tokens`'ı `.invoke()` metodunun bir parametresi olarak ayarlayın.

In [14]:
# Set the maximum number of output tokens to 200

model.max_output_tokens = 200

# Set the temperature to 1.0

model.temperature = 1.0

# Generate a response with the new settings

response = model.invoke("Write a social media post about how much you're learning about transformers")

# Display the response

Markdown(response.content)


Here are a few options for a social media post about learning about transformers, with slightly different tones. Choose the one that best fits your style!

**Option 1: Enthusiastic & Slightly Technical**

> Dive headfirst into the world of AI and guess what's blowing my mind? ✨ **Transformers!** 🤯 Seriously, the architecture behind these models is a game-changer. I'm deep in the rabbit hole of attention mechanisms, positional encoding, and encoder-decoder stacks. The ability to process sequential data like this is revolutionary. So much to learn, but loving every minute! #AI #MachineLearning #DeepLearning #Transformers #NLP #Tech

**Option 2: Casual & Wonderstruck**

> My brain is officially buzzing with new knowledge! 🧠 I've been spending a lot of time learning about **Transformers**, and WOW. It's like unlocking a new level of understanding for how AI processes language and so much more. Mind officially blown by the elegance of these models. Can't wait to see what else I discover! 🤓 #Learning #AI #TechJourney #TransformersModel #FutureTech

**Option 3: Focused on Application & Impact**

> Lately, I've been fascinated by the power of **Transformers** in AI. From generating creative text to understanding complex queries, their impact is undeniable. 🚀 I'm really enjoying diving into the intricacies of their design and how they're reshaping fields like Natural Language Processing. So much potential! #ArtificialIntelligence #MachineLearning #TransformersAI #Innovation #TechTrends

**Option 4: Short & Sweet**

> Officially on a deep dive into **Transformers** and my mind is expanding! 🤯 So much to absorb, so much to learn. #AI #Transformers #Learning

**Remember to also consider adding:**

*   **A relevant image or GIF:** A brain graphic, a circuit board, a futuristic image, or even a relevant meme can make your post more engaging.
*   **A question to encourage interaction:** "What's the most mind-blowing thing you've learned about AI recently?" or "Any favorite resources for learning about Transformers?"

Choose the one that feels most authentic to you and happy learning!

Bunun avantajı? Bu LangChain Chat Model birçok başka API'yi destekleyebilir.

Başka bir modele geçmek için değiştirmeniz gereken tek şeyler:
1. Diğer model için bir API anahtarı alın ve kodunuzda tanımlayın.
2. Modeli başlatırken model ve sağlayıcıyı değiştirin.

### Çoklu Mesajlar

`.invoke()` fonksiyonunu sadece tek bir mesajla kullanmak biraz kısıtlayıcı.

Şu gibi birden fazla mesaj sağlayabilirsiniz:
- `SystemMessage` veya sistem mesajları: modelin nasıl davranacağını söylemek için
- `HumanMessage` veya Kullanıcı mesajları: kullanıcıdan gelen girdi
- `AIMessage` veya Asistan mesajları: modelden gelen yanıt

Bir sosyal medya yazarı yapalım.

Modele nasıl davranacağını açıklayan bir sistem mesajı göndereceğiz. Sonra kullanıcı mesajında, kendimizi sadece yazacağı konuyu vermekle sınırlayabiliriz.

Bunu nasıl yapacağınızı öğrenmek için [LangChain'in "Messages" dokümantasyonu](https://docs.langchain.com/oss/python/langchain/messages)'na bakın.

Sistem mesajı için ilhama mı ihtiyacınız var? İşte başlamanız için temel bir talimat:

```python
"""Sen Üretken AI öğrencisi için gönderiler yazan yaratıcı bir sosyal medya yazarısın.
Gönderilerinde her zaman kelime oyunu ve harekete geçirici çağrı bulunur.
Gönderilerin maksimum 200 karakter uzunluğundadır.
Her zaman emoji kullanırsın.
"""
```

In [15]:
# Import the necessary classes

from langchain_core.messages import HumanMessage, SystemMessage

# Create a list of messages

messages = [
    SystemMessage(
        """You are a creative social media writer writing posts for a Gen AI student.
        Your posts always include a pun, and a call to action.
        Your posts are maximum 200 characters long.
        You always use emojis.
        """
        ),
    HumanMessage("I'm learning about transformers."),
]

# Generate a response using the list of messages

response = model.invoke(messages)

# Display the response

Markdown(response.content)


Get ready to be **transformed**! 🤖 Dive into the amazing world of transformers and unlock your AI potential. 🚀 What will you build first? ✨ #AI #Transformers #Tech

🏁 Tebrikler! Artık LangChain kullanarak çoklu mesajlarla temel prompt yazma konusunda uzmanlaştınız.